In [1]:
import pandas as pd

In [67]:
df = pd.read_csv("E:/!!Research/!!!Data/graph_analysis/lek_data/lek_data_binary_cleaned.csv")
df.head()

,lek_id,year,activity,x_easting,y_northing
0,2,2007,True,672527,3730500
1,2,2008,True,672527,3730500
2,2,2009,True,672527,3730500
3,2,2010,True,672527,3730500
4,4,2007,True,669625,3729786


In [68]:
# Define the range of years you want
years = range(df['year'].min(), df['year'].max() + 1)

# Create a complete DataFrame for all leks and years
leks = df['lek_id'].unique()
complete_index = pd.MultiIndex.from_product([leks, years], names=['lek_id', 'year'])
complete_df = pd.DataFrame(index=complete_index).reset_index()

# Check the columns in the original DataFrame
#print("Original DataFrame columns:", df.columns)

# Create a unique DataFrame to maintain site info and coordinates
unique_sites_df = df[['lek_id', 'x_easting', 'y_northing']].drop_duplicates()

# Merge the unique sites DataFrame with the complete DataFrame
result_df = pd.merge(complete_df, unique_sites_df, on='lek_id', how='left')

# Now merge the activity data
df = pd.merge(result_df, df[['lek_id', 'year', 'activity']], on=['lek_id', 'year'], how='left')

# Replace NaN values in the 'activity' column with "not surveyed"
df['activity'] = df['activity'].fillna('not surveyed')

# Display the result
print(df)

       lek_id  year  x_easting  y_northing      activity
0           2  1971     672527     3730500  not surveyed
1           2  1972     672527     3730500  not surveyed
2           2  1973     672527     3730500  not surveyed
3           2  1974     672527     3730500  not surveyed
4           2  1975     672527     3730500  not surveyed
...       ...   ...        ...         ...           ...
26587   99999  2014     649863     3714293         False
26588   99999  2015     649863     3714293  not surveyed
26589   99999  2016     649863     3714293  not surveyed
26590   99999  2017     649863     3714293  not surveyed
26591   99999  2018     649863     3714293  not surveyed

[26592 rows x 5 columns]


In [65]:
# Convert 'activity' from string to boolean
#df['activity'] = df['activity'].map({'True': True, 'False': False})

# Sort by 'lek_id' and 'year' to ensure proper cumulative and rolling calculation
df = df.sort_values(by=['lek_id', 'year']).reset_index(drop=True)

# Create active_last_5_years column
def calculate_active_status(group):
    # Rolling maximum and shift to exclude the current year
    active_status = group['activity'].rolling(window=5, min_periods=1).max().shift()
    return active_status.fillna(False)

# Apply the function to each group
df['active_last_5_years'] = df.groupby('lek_id', group_keys=False).apply(calculate_active_status)

# Convert to 'TRUE'/'FALSE' for readability
df['active_last_5_years'] = df['active_last_5_years'].map({True: 'TRUE', False: 'FALSE'})

# Count active years in the last 5 years, shifted to exclude the current year
def count_active_years(group):
    return group['activity'].rolling(window=5, min_periods=1).sum().shift().fillna(0).astype(int)

# Apply the function to each group for counting
df['active_count_last_5_years'] = df.groupby('lek_id', group_keys=False).apply(count_active_years)

print(df)

       lek_id  year  x_easting  y_northing activity active_last_5_years  \
0           2  1971     672527     3730500      NaN               FALSE   
1           2  1972     672527     3730500      NaN               FALSE   
2           2  1973     672527     3730500      NaN               FALSE   
3           2  1974     672527     3730500      NaN               FALSE   
4           2  1975     672527     3730500      NaN               FALSE   
...       ...   ...        ...         ...      ...                 ...   
26587   99999  2014     649863     3714293    False                TRUE   
26588   99999  2015     649863     3714293      NaN                TRUE   
26589   99999  2016     649863     3714293      NaN                TRUE   
26590   99999  2017     649863     3714293      NaN                TRUE   
26591   99999  2018     649863     3714293      NaN               FALSE   

       active_count_last_5_years  
0                              0  
1                            

C:\Users\Justin\AppData\Local\Temp\ipykernel_16892\4085168194.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['active_last_5_years'] = df.groupby('lek_id', group_keys=False).apply(calculate_active_status)
C:\Users\Justin\AppData\Local\Temp\ipykernel_16892\4085168194.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['active_count_last_5_years'] = df.groupby('lek_id', group_keys=False).apply(cou

In [66]:
df.to_csv("E:/!!Research/!!!Data/graph_analysis/lek_data/lek_data_binary_rolling_activity.csv")